In [2]:
betapro_partners = [
# TBD
 ]

In [2]:
 import requests
 import pandas as pd
 from xml.etree import ElementTree as ET
 from time import sleep
 from datetime import datetime


 def fetch_betapro_data(spreadsheet_id: str):
     """
     Получает данные из API Betapro и добавляет их в Google Sheets.

     :param spreadsheet_id: Идентификатор Google Sheets.
     """
     def get_xml_data(partner_id: str, password: str):
         """
         Отправляет запрос в API Betapro и получает XML-данные.
         """
         url = "http://api.betapro.ru:8080/bp/hs/wsrv"
         xml_body = f'<request partner_id="{partner_id}" password="{password}" request_type="151"></request>'
         headers = {
             "Content-Type": "text/xml",
             "Content-Length": str(len(xml_body)),
             "Connection": "close"
         }

         response = requests.post(url, headers=headers, data=xml_body, timeout=60, verify=False)
         if response.status_code != 200:
             raise Exception(f"Ошибка API Betapro: {response.status_code}, {response.text}")
         return response.text

     def parse_xml_to_dataframe(xml_data: str, partner_id: str) -> pd.DataFrame:
         """
         Парсит XML-ответ API Betapro в DataFrame.
         """
         root = ET.fromstring(xml_data)
         ns = {'ns': 'FBox/XMLSchema'}
         data = []

         # Парсинг <good>
         for good in root.findall('ns:good', ns):
             data.append(good.attrib)

         # Парсинг <doc>
         for doc in root.findall('ns:doc', ns):
             doc_data = {
                 "doc_type": doc.get("doc_type"),
                 "doc_type_descrip": doc.get("doc_type_descrip"),
                 "doc_date": doc.get("doc_date"),
                 "doc_datetime": doc.get("doc_datetime"),
                 "zdoc_id": doc.get("zdoc_id"),
                 "partner_id": partner_id
             }
             data.append(doc_data)

         df = pd.DataFrame(data)

         # Если есть данные о запасах (qnt), добавляем расчет "Свободного остатка"
         if "qnt" in df.columns:
             qt_len = len([col for col in df.columns if col.startswith("qnt")])
             df["Свободный остаток"] = df.apply(
                 lambda row: float(row["qnt"]) - sum([float(row.get(f"qnt{i}", 0)) for i in range(2, qt_len)]),
                 axis=1
             )
         df["Дата Изменения"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
         return df

     # Список партнеров из config
     partners = betapro_partners
     if not partners:
         raise Exception("Партнеры Betapro не указаны в конфигурации.")

     df_list = []
     for partner in partners:
         partner_id = partner["partner_id"]
         password = partner["password"]

         print(f"Загрузка данных для партнера {partner_id}...")
         xml_data = get_xml_data(partner_id, password)
         df = parse_xml_to_dataframe(xml_data, partner_id)
         df_list.append(df)
         sleep(3)  # Задержка между запросами

     # Объединение данных всех партнеров
     final_df = pd.concat(df_list, ignore_index=True)

     # Удаляем дубликаты
     final_df = final_df.drop_duplicates()
     return final_df

In [3]:
 try:
     df = fetch_betapro_data("1Gjx1UFXW0nZPUiDIrzNedBiJad1B__YQmTjH0uTSnws")
     print("Тест успешно завершен.")
 except Exception as e:
     print(f"Произошла ошибка: {e}")

Загрузка данных для партнера 1040...
Загрузка данных для партнера 1134...
Загрузка данных для партнера 1160...
Загрузка данных для партнера 1159...
Тест успешно завершен.


In [4]:
df.shape

(180, 12)

In [12]:
df

,good_id,qual_type,qnt,qnt2,qnt3,qnt4,qnt5,qnt6,qnt7,qnt8,Свободный остаток,Дата Изменения
0,778988312353,1,0,0,0,0,0,0,0,1,0.0,2024-12-19 23:37:55
1,778988396926,1,5,0,0,0,0,0,0,5,5.0,2024-12-19 23:37:55
2,887961813067,1,0,0,0,0,0,0,0,1,0.0,2024-12-19 23:37:55
3,2000290429022,1,1,0,0,0,1,0,0,0,0.0,2024-12-19 23:37:55
4,4027093375255,1,0,0,0,0,0,0,0,1,0.0,2024-12-19 23:37:55
...,...,...,...,...,...,...,...,...,...,...,...,...
172,4673737630770,1,43,0,0,0,0,0,0,0,43.0,2024-12-19 23:38:05
173,4673737630787,1,190,0,0,0,0,0,0,0,190.0,2024-12-19 23:38:05
174,4673737630794,1,3,0,0,0,0,0,0,0,3.0,2024-12-19 23:38:05
175,4673737630824,1,12,0,0,0,0,0,0,0,12.0,2024-12-19 23:38:05


In [7]:
df.good_id.nunique()

136

In [8]:
df.drop_duplicates().shape

(180, 12)

In [9]:
df.drop_duplicates(subset=['good_id', 'qual_type', 'qnt', 'qnt2', 'qnt3', 'qnt4', 'qnt5', 'qnt6', 'qnt7', 'qnt8', 'Свободный остаток']).shape

(180, 12)

In [10]:
df.drop_duplicates(subset=['good_id', 'qual_type', 'qnt', 'Свободный остаток']).shape

(180, 12)

In [11]:
df.drop_duplicates(subset=['good_id', 'qual_type', 'Свободный остаток']).shape

(180, 12)

In [12]:
df.drop_duplicates(subset=['good_id', 'Свободdный остаток']).shape

(179, 12)

In [6]:
import requests

In [7]:
     def get_xml_data(partner_id: str, password: str):
         """
         Отправляет запрос в API Betapro и получает XML-данные.
         """
         url = "http://api.betapro.ru:8080/bp/hs/wsrv"
         xml_body = f'<request partner_id="{partner_id}" password="{password}" request_type="151"></request>'
         headers = {
             "Content-Type": "text/xml",
             "Content-Length": str(len(xml_body)),
             "Connection": "close"
         }

         response = requests.post(url, headers=headers, data=xml_body, timeout=60, verify=False)
         if response.status_code != 200:
             raise Exception(f"Ошибка API Betapro: {response.status_code}, {response.text}")
         return response.text

In [8]:
password = ""

In [11]:
partner_id = "1040"

In [12]:
data = get_xml_data(partner_id, password)

In [13]:
data

'<response xmlns="FBox/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:type="response151" state="0">\n  <good good_id="778988312353" qual_type="1" qnt="0" qnt2="0" qnt3="0" qnt4="0" qnt5="0" qnt6="0" qnt7="0" qnt8="1"/>\n  <good good_id="778988396926" qual_type="1" qnt="5" qnt2="0" qnt3="0" qnt4="0" qnt5="0" qnt6="0" qnt7="0" qnt8="5"/>\n  <good good_id="887961813067" qual_type="1" qnt="0" qnt2="0" qnt3="0" qnt4="0" qnt5="0" qnt6="0" qnt7="0" qnt8="1"/>\n  <good good_id="2000290429022" qual_type="1" qnt="1" qnt2="0" qnt3="0" qnt4="0" qnt5="1" qnt6="0" qnt7="0" qnt8="0"/>\n  <good good_id="4027093375255" qual_type="1" qnt="0" qnt2="0" qnt3="0" qnt4="0" qnt5="0" qnt6="0" qnt7="0" qnt8="1"/>\n  <good good_id="4027093375255" qual_type="2" qnt="0" qnt2="0" qnt3="1" qnt4="0" qnt5="0" qnt6="0" qnt7="0" qnt8="0"/>\n  <good good_id="4607809990110" qual_type="1" qnt="0" qnt2="0" qnt3="0" qnt4="0" qnt5="0" qnt6="0" qnt7="0" qnt8="10"/>\n  <good good_id="4609639568033" qual_ty